# **Import necessary library, especially Faker**
We will try to randomize/generate/dummy transaction data for Transjakarta based on publicly accessed Master Data.

In [ ]:
import pandas as pd
import numpy as np
import random as rd
import calendar
import datetime as dt
from datetime import datetime, timedelta

from faker import Faker
from faker.providers import BaseProvider

In [ ]:
pd.set_option('display.max_columns',None)

# **First, we load master data**

In [54]:
dfCorridor = pd.read_csv('routes.csv')
dfCorridor

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order
0,1,Tije,1,Blok M - Kota,BRT,3,NaN,D62126,FFFFFF,NaN
1,2,Tije,2,Pulo Gadung - Monas,BRT,3,NaN,2F489C,FFFFFF,NaN
2,3,Tije,3,Kalideres - Monas via Veteran,BRT,3,NaN,FDCB1C,000000,NaN
3,4,Tije,4,Pulo Gadung - Galunggung,BRT,3,NaN,512C62,FFFFFF,NaN
4,5,Tije,5,Kampung Melayu - Ancol,BRT,3,NaN,D46425,000000,NaN
...,...,...,...,...,...,...,...,...,...,...
239,8M,Tije,8M,Tanjung Duren - Tanah Abang,Angkutan Umum Integrasi,3,NaN,DD6AAB,000000,NaN
240,BW2,Tije,BW2,Monas Explorer,Bus Wisata,3,NaN,FFB6DB,000000,NaN
241,T21,Tije,T21,Taman Banjar Wijaya - Petamburan,Royaltrans,3,NaN,9C4782,FFFFFF,NaN
242,8N,Tije,8N,Kebayoran - Petamburan via Asia Afrika,Angkutan Umum Integrasi,3,NaN,dd6aab,000000,NaN


In [55]:
dfTrips = pd.read_csv('trips.csv')
dfTrips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,trip_short_name
0,10A,SH,10A-L01,Tanjung Priok dan Rusun Marunda,0,NaN,10A-L01_shp,Rusun Marunda - Tanjung Priok - Rusun Marunda
1,10B,SH,10B-R01,Halim,0,NaN,10B-R01_shp,Rusun Cipinang Besar Selatan - Halim
2,10B,SH,10B-R02,Rusun Cipinang Besar Selatan,1,NaN,10B-R02_shp,Halim - Rusun Cipinang Besar Selatan
3,10D,SH,10D-R01,Kampung Rambutan,0,NaN,10D-R01_shp,Tanjung Priok - Kampung Rambutan
4,10D,SH,10D-R02,Tanjung Priok,1,NaN,10D-R02_shp,Kampung Rambutan - Tanjung Priok
...,...,...,...,...,...,...,...,...
579,JAK.73,SH,JAK.73-R06,Jambore,1,NaN,1cfx,Pasar Rebo - Jambore
580,JAK.25,SH,JAK.25-R04,Kalisari,0,NaN,5kyh,Pasar Rebo - Kalisari
581,JAK.25,SH,JAK.25-R06,Pasar Rebo,1,NaN,shsi,Kalisari - Pasar Rebo
582,JAK.28,SH,JAK.28-R05,Cibubur,0,NaN,l710,Pasar Rebo - Cibubur


In [56]:
series = dfTrips['route_id'].value_counts()
selectIndex = series[series == 1].index
selectIndex

Index(['13B', '1T', '11W', '11M', '7D', '1F', '12F', 'S12', '8N', '6W', '2F',
       '3F', 'BW9', 'BW4', 'BW2', 'BW1', 'B21', '9N', '8M', '8K', '8C', '7W',
       '7V', '7U', '7T', '7R', '7P', '7C', '6T', '6Q', '6K', '6C', '5F', '5B',
       '4D', '10A'],
      dtype='object', name='route_id')

In [57]:
dfTrips.drop(dfTrips[dfTrips['route_id'].isin(selectIndex)].index, inplace=True)
dfTrips


,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,trip_short_name
1,10B,SH,10B-R01,Halim,0,NaN,10B-R01_shp,Rusun Cipinang Besar Selatan - Halim
2,10B,SH,10B-R02,Rusun Cipinang Besar Selatan,1,NaN,10B-R02_shp,Halim - Rusun Cipinang Besar Selatan
3,10D,SH,10D-R01,Kampung Rambutan,0,NaN,10D-R01_shp,Tanjung Priok - Kampung Rambutan
4,10D,SH,10D-R02,Tanjung Priok,1,NaN,10D-R02_shp,Kampung Rambutan - Tanjung Priok
5,10H,SH,10H-L04,Bundaran Senayan dan Tanjung Priok,0,NaN,zc5r,Tanjung Priok - Bundaran Senayan - Tanjung Priok
...,...,...,...,...,...,...,...,...
579,JAK.73,SH,JAK.73-R06,Jambore,1,NaN,1cfx,Pasar Rebo - Jambore
580,JAK.25,SH,JAK.25-R04,Kalisari,0,NaN,5kyh,Pasar Rebo - Kalisari
581,JAK.25,SH,JAK.25-R06,Pasar Rebo,1,NaN,shsi,Kalisari - Pasar Rebo
582,JAK.28,SH,JAK.28-R05,Cibubur,0,NaN,l710,Pasar Rebo - Cibubur


In [58]:
# Group the DataFrame by 'route_id' and filter groups that don't have both 0 and 1 in 'direction_id'
filteredGroups = dfTrips.groupby('route_id').filter(lambda x: set(x['direction_id']) != {0, 1})

# Get the unique 'route_id' values from the filtered groups
selectRoute = filteredGroups['route_id'].unique()
selectRoute


array(['12A', '1B', '1R', '5M', '5', '6V', '8D', '9E', '9H', 'B11', 'D31',
       'JAK.17', '8A', '1A'], dtype=object)

In [59]:
dfTrips[dfTrips['route_id'].isin(selectRoute)]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,trip_short_name
25,12A,SH,12A-L02,Kota dan Kali Adem,0,NaN,12A-L02_shp,Kali Adem - Kota - Kali Adem
26,12A,SH,12A-R03,Kota,0,NaN,12A-R03_shp,Kali Adem - Kota
59,1B,XSH_1B,1B-L01,Tosari dan Stasiun Palmerah,0,NaN,1B-L01_shp,Stasiun Palmerah - Tosari - Stasiun Palmerah
94,1R,SH,1R-L03,Explorer Tanah Abang dan Senen,0,NaN,1R-L03_shp,Senen - Explorer Tanah Abang - Senen
95,1R,SH,1R-L04,Tanah Abang dan Senen,0,NaN,1R-L04_shp,Senen - Tanah Abang - Senen
167,5M,SH,5M-L03,Explorer Tanah Abang dan Kampung Melayu,0,NaN,5M-L03_shp,Kampung Melayu - Explorer Tanah Abang - Kampun...
168,5M,SH,5M-L04,Explorer Tanah Abang dan Kampung Melayu via Ci...,0,NaN,5M-L04_shp,Kampung Melayu - Explorer Tanah Abang - Kampun...
169,5M,SH,5M-L05,Tugu Tani via Pejambon dan Kampung Melayu,0,NaN,5M-L05_shp,Kampung Melayu - Tugu Tani via Pejambon - Kamp...
170,5M,SH,5M-L06,Tanah Abang dan Kampung Melayu,0,NaN,doln,Kampung Melayu - Tanah Abang - Kampung Melayu
171,5M,SH,5M-P21,Jati Baru dan Kampung Melayu,0,NaN,5M-P21_shp,Kampung Melayu - Jati Baru - Kampung Melayu


In [60]:
dfTrips.loc[dfTrips['trip_id'] == 'M7-R03', 'direction_id'] = 0
dfTrips.loc[dfTrips['trip_id'] == 'M8-R03', 'direction_id'] = 1
display(dfTrips[dfTrips['route_id'].isin(selectRoute)],
        dfTrips)


,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,trip_short_name
25,12A,SH,12A-L02,Kota dan Kali Adem,0,NaN,12A-L02_shp,Kali Adem - Kota - Kali Adem
26,12A,SH,12A-R03,Kota,0,NaN,12A-R03_shp,Kali Adem - Kota
59,1B,XSH_1B,1B-L01,Tosari dan Stasiun Palmerah,0,NaN,1B-L01_shp,Stasiun Palmerah - Tosari - Stasiun Palmerah
94,1R,SH,1R-L03,Explorer Tanah Abang dan Senen,0,NaN,1R-L03_shp,Senen - Explorer Tanah Abang - Senen
95,1R,SH,1R-L04,Tanah Abang dan Senen,0,NaN,1R-L04_shp,Senen - Tanah Abang - Senen
167,5M,SH,5M-L03,Explorer Tanah Abang dan Kampung Melayu,0,NaN,5M-L03_shp,Kampung Melayu - Explorer Tanah Abang - Kampun...
168,5M,SH,5M-L04,Explorer Tanah Abang dan Kampung Melayu via Ci...,0,NaN,5M-L04_shp,Kampung Melayu - Explorer Tanah Abang - Kampun...
169,5M,SH,5M-L05,Tugu Tani via Pejambon dan Kampung Melayu,0,NaN,5M-L05_shp,Kampung Melayu - Tugu Tani via Pejambon - Kamp...
170,5M,SH,5M-L06,Tanah Abang dan Kampung Melayu,0,NaN,doln,Kampung Melayu - Tanah Abang - Kampung Melayu
171,5M,SH,5M-P21,Jati Baru dan Kampung Melayu,0,NaN,5M-P21_shp,Kampung Melayu - Jati Baru - Kampung Melayu


,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,trip_short_name
1,10B,SH,10B-R01,Halim,0,NaN,10B-R01_shp,Rusun Cipinang Besar Selatan - Halim
2,10B,SH,10B-R02,Rusun Cipinang Besar Selatan,1,NaN,10B-R02_shp,Halim - Rusun Cipinang Besar Selatan
3,10D,SH,10D-R01,Kampung Rambutan,0,NaN,10D-R01_shp,Tanjung Priok - Kampung Rambutan
4,10D,SH,10D-R02,Tanjung Priok,1,NaN,10D-R02_shp,Kampung Rambutan - Tanjung Priok
5,10H,SH,10H-L04,Bundaran Senayan dan Tanjung Priok,0,NaN,zc5r,Tanjung Priok - Bundaran Senayan - Tanjung Priok
...,...,...,...,...,...,...,...,...
579,JAK.73,SH,JAK.73-R06,Jambore,1,NaN,1cfx,Pasar Rebo - Jambore
580,JAK.25,SH,JAK.25-R04,Kalisari,0,NaN,5kyh,Pasar Rebo - Kalisari
581,JAK.25,SH,JAK.25-R06,Pasar Rebo,1,NaN,shsi,Kalisari - Pasar Rebo
582,JAK.28,SH,JAK.28-R05,Cibubur,0,NaN,l710,Pasar Rebo - Cibubur


In [61]:
dfStopTimes = pd.read_csv('stop_times.csv')
dfStopTimes

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,shape_dist_traveled,stop_headsign,timepoint
0,10B-R01,05:00:00,05:00:10,B03270P,0,0.0,NaN,NaN,0
1,10B-R01,05:01:34,05:01:44,B05785P,1,0.0,NaN,NaN,0
2,10B-R01,05:02:59,05:03:09,B05536P,2,0.0,NaN,NaN,0
3,10B-R01,05:06:48,05:06:58,B02723P,3,0.0,NaN,NaN,0
4,10B-R02,05:00:00,05:00:10,B02723P,0,0.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
18498,JAK.73-R05,05:36:20,05:36:30,B00305P,26,0.0,NaN,NaN,0
18499,JAK.73-R05,05:38:55,05:39:05,B04633P,27,0.0,NaN,NaN,0
18500,JAK.73-R05,05:40:16,05:40:26,B02188P,28,0.0,NaN,NaN,0
18501,JAK.73-R05,05:42:32,05:42:42,B04705P,29,0.0,NaN,NaN,0


In [62]:
dfStops = pd.read_csv('stops.csv')
dfStops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,wheelchair_boarding,parent_station,platform_code
0,B00001P,NaN,18 Office Park,NaN,-6.299146,106.832100,1,NaN,0,2,NaN,NaN
1,B00002P,NaN,ABA,NaN,-6.194149,106.839390,1,NaN,0,2,NaN,NaN
2,B00003P,NaN,Acacia Residence,NaN,-6.263358,106.756750,1,NaN,0,2,NaN,NaN
3,B00004P,NaN,ACC Simatupang,NaN,-6.304475,106.848580,1,NaN,0,2,NaN,NaN
4,B00005P,NaN,ACE Hardware,NaN,-6.387532,106.827380,1,NaN,0,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8023,G00743,NaN,Transport Hub Dukuh Atas,NaN,-6.201825,106.823823,1,NaN,0,2,B07663P,NaN
8024,G00744,NaN,Transport Hub Dukuh Atas,NaN,-6.201821,106.824006,1,NaN,0,2,B07663P,NaN
8025,B07657P,NaN,Sekolah Al Wahyu Cibubur,NaN,-6.365855,106.887330,1,NaN,0,2,NaN,NaN
8026,B04612P,NaN,Akses Bawah Flyover Raya Bogor 2,NaN,-6.306687,106.864709,1,NaN,0,2,NaN,NaN


In [63]:
dfFareRules = pd.read_csv('fare_rules.csv')
dfFareRules

,fare_id,route_id,origin_id,destination_id,contains_id
0,FP,1,NaN,NaN,NaN
1,FP,2,NaN,NaN,NaN
2,FP,3,NaN,NaN,NaN
3,FP,4,NaN,NaN,NaN
4,FP,5,NaN,NaN,NaN
...,...,...,...,...,...
239,FP,8M,NaN,NaN,NaN
240,GR,BW2,NaN,NaN,NaN
241,PP3,T21,NaN,NaN,NaN
242,FP,8N,NaN,NaN,NaN


In [64]:
dfFareAttr = pd.read_csv('fare_attributes.csv')
dfFareAttr

,fare_id,price,currency_type,payment_method,transfers,agency_id,transfer_duration
0,FP,3500,IDR,0,NaN,Tije,10800
1,PP,20000,IDR,0,NaN,Tije,10800
2,GR,0,IDR,0,NaN,Tije,10800
3,FP2,3500,IDR,0,1.0,Tije,10800
4,PP2,25000,IDR,0,NaN,Tije,10800
5,PP3,35000,IDR,0,NaN,Tije,10800


# **Merging all tables with necessary columns as complete relational data**

In [65]:
dfRoutes = dfStopTimes[['trip_id','stop_id','stop_sequence']].merge(dfStops[['stop_id','stop_name','stop_lat','stop_lon']],
                                                                   'left',
                                                                   'stop_id').merge(dfTrips[['trip_id','trip_headsign','direction_id','route_id']],
                                                                                    'left',
                                                                                    'trip_id').merge(dfCorridor[['route_id','route_long_name']],
                                                                                                     'left',
                                                                                                     'route_id').merge(dfFareRules[['route_id','fare_id']],
                                                                                                                       'left',
                                                                                                                       'route_id').merge(dfFareAttr[['fare_id','price']],
                                                                                                                                         'left',
                                                                                                                                         'fare_id')
dfRoutes = dfRoutes[['route_id','fare_id','price','route_long_name','direction_id','trip_id','trip_headsign','stop_id','stop_sequence','stop_name','stop_lat','stop_lon']].sort_values(['route_id','trip_id','stop_sequence','direction_id'])
dfRoutes

,route_id,fare_id,price,route_long_name,direction_id,trip_id,trip_headsign,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
10591,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,P00017,0,Blok M,-6.243312,106.801750
10592,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,G00012,1,Asean,-6.239911,106.798970
10593,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,G00009,2,Masjid Agung,-6.236472,106.798412
10594,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,G00696,3,Bundaran Senayan,-6.227766,106.800954
10595,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,G00016,4,Gelora Bung Karno,-6.224137,106.805837
...,...,...,...,...,...,...,...,...,...,...,...,...
9649,NaN,NaN,NaN,NaN,NaN,S12-L04,NaN,B05701P,0,Term. BSD,-6.306375,106.685360
9650,NaN,NaN,NaN,NaN,NaN,S12-L04,NaN,B05488P,1,St. MRT Fatmawati,-6.292589,106.793600
9651,NaN,NaN,NaN,NaN,NaN,S12-L04,NaN,B03177P,2,RS Siloam,-6.292071,106.784670
9652,NaN,NaN,NaN,NaN,NaN,S12-L04,NaN,B00229P,3,Carrefour Lebak Bulus,-6.287529,106.776800


In [66]:
dfRoutes[dfRoutes['price'].isna()]

,route_id,fare_id,price,route_long_name,direction_id,trip_id,trip_headsign,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
7370,NaN,NaN,NaN,NaN,NaN,10A-L01,NaN,B03282P,0,Rusun Marunda,-6.097099,106.96231
7371,NaN,NaN,NaN,NaN,NaN,10A-L01,NaN,B03283P,1,Rusun Marunda 1,-6.098204,106.96232
7372,NaN,NaN,NaN,NaN,NaN,10A-L01,NaN,B05078P,2,Simpang Pitungan,-6.100212,106.96060
7373,NaN,NaN,NaN,NaN,NaN,10A-L01,NaN,B06039P,3,STIP 2,-6.104694,106.95903
7374,NaN,NaN,NaN,NaN,NaN,10A-L01,NaN,B00136P,4,Bidara 1,-6.106097,106.95950
...,...,...,...,...,...,...,...,...,...,...,...,...
9649,NaN,NaN,NaN,NaN,NaN,S12-L04,NaN,B05701P,0,Term. BSD,-6.306375,106.68536
9650,NaN,NaN,NaN,NaN,NaN,S12-L04,NaN,B05488P,1,St. MRT Fatmawati,-6.292589,106.79360
9651,NaN,NaN,NaN,NaN,NaN,S12-L04,NaN,B03177P,2,RS Siloam,-6.292071,106.78467
9652,NaN,NaN,NaN,NaN,NaN,S12-L04,NaN,B00229P,3,Carrefour Lebak Bulus,-6.287529,106.77680


In [67]:
dfRoutes.dropna(subset='route_id',inplace=True)
dfRoutes

,route_id,fare_id,price,route_long_name,direction_id,trip_id,trip_headsign,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
10591,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,P00017,0,Blok M,-6.243312,106.801750
10592,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,G00012,1,Asean,-6.239911,106.798970
10593,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,G00009,2,Masjid Agung,-6.236472,106.798412
10594,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,G00696,3,Bundaran Senayan,-6.227766,106.800954
10595,1,FP,3500.0,Blok M - Kota,0.0,1-P23,Tosari dan Blok M,G00016,4,Gelora Bung Karno,-6.224137,106.805837
...,...,...,...,...,...,...,...,...,...,...,...,...
15727,T21,PP3,35000.0,Taman Banjar Wijaya - Petamburan,0.0,T21-R01,Petamburan,B06908P,3,Slipi Petamburan 2,-6.201683,106.800020
15728,T21,PP3,35000.0,Taman Banjar Wijaya - Petamburan,1.0,T21-R02,Taman Banjar Wijaya,B06909P,0,Slipi Petamburan 3,-6.200982,106.799123
15729,T21,PP3,35000.0,Taman Banjar Wijaya - Petamburan,1.0,T21-R02,Taman Banjar Wijaya,B07584P,1,Plaza Slipi Jaya,-6.188705,106.796756
15730,T21,PP3,35000.0,Taman Banjar Wijaya - Petamburan,1.0,T21-R02,Taman Banjar Wijaya,B06937P,2,RS Harapan Kita 1,-6.184915,106.796638


In [68]:
dfRoutes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17459 entries, 10591 to 15731
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   route_id         17459 non-null  object 
 1   fare_id          17459 non-null  object 
 2   price            17459 non-null  float64
 3   route_long_name  17459 non-null  object 
 4   direction_id     17459 non-null  float64
 5   trip_id          17459 non-null  object 
 6   trip_headsign    17459 non-null  object 
 7   stop_id          17459 non-null  object 
 8   stop_sequence    17459 non-null  int64  
 9   stop_name        17459 non-null  object 
 10  stop_lat         17459 non-null  float64
 11  stop_lon         17459 non-null  float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.7+ MB


In [74]:
dfRoutes.to_csv('dfRoute.csv')

# **Creating Master Customers with FAKER library**

In [69]:
fake = Faker(locale='id_ID')
fake

## **Use .simple_profile() to generate fake/dummy profile**
And shuffle it so the profile composition is randomize when generating the transaction.

In [70]:
payCardIDTable = []
payCardNameTable = []
payCardSexTable = []
payCardBirthDate = []

for _ in range(1,2001,1) :
    payCardIDTable.append(fake.credit_card_number())
    if (_ >= 1 and _ <= 392) or (_ >= 951 and _ <= 1197) or (_ >= 1301 and _ <= 1342) or (_ >= 1351 and _ <= 1602) :
        profile = fake.simple_profile(sex='M')
        payCardNameTable.append(profile['name'])
        payCardSexTable.append(profile['sex'])
    elif (_ >= 393 and _ <= 950) or (_ >= 1198 and _ <= 1300) or (_ >= 1343 and _ <= 1350) or (_ >= 1603 and _ <= 2000) :
        profile = fake.simple_profile(sex='F')
        payCardNameTable.append(profile['name'])
        payCardSexTable.append(profile['sex'])
    if _ >= 1 and _ <= 950 : # 950 first profile is gen Y
        payCardBirthDate.append(fake.date_between(start_date = dt.date(1981,1,1),
                                                  end_date = dt.date(1996,12,31)).year)
    elif _ >= 951 and _ <= 1300 : # 350 next profile is Gen X
        payCardBirthDate.append(fake.date_between(start_date = dt.date(1965,1,1),
                                                  end_date = dt.date(1980,12,31)).year)
    elif _ >= 1301 and _ <= 1350 : # 50 next profile is Baby Boomers
        payCardBirthDate.append(fake.date_between(start_date = dt.date(1946,1,1),
                                                  end_date = dt.date(1964,12,31)).year)
    elif _ >= 1351 and _ <= 2000 : # 650 next profile is Gen Z
        payCardBirthDate.append(fake.date_between(start_date = dt.date(1997,1,1),
                                                  end_date = dt.date(2012,12,31)).year)

dfPayCards = pd.DataFrame(zip(payCardIDTable,payCardNameTable,payCardSexTable,payCardBirthDate),
                         columns=['payCardIDTable','payCardNameTable','payCardSexTable','payCardBirthDate'])
dfPayCards = dfPayCards.sample(frac=1,
                               random_state=99).reset_index(drop=True)
dfPayCards

,payCardIDTable,payCardNameTable,payCardSexTable,payCardBirthDate
0,4368520028973555,"H. Asman Prakasa, M.Farm",M,2011
1,5343809282239143,"Mahesa Pratiwi, S.Sos",F,1999
2,213181258600684,Hj. Tari Narpati,F,1990
3,4616141698869,Jaiman Siregar,F,1965
4,501860613964,Syahrini Nugroho,M,1988
...,...,...,...,...
1995,3554847388768390,Pardi Maheswara,F,2010
1996,4303534664855057,Sutan Setya Marpaung,M,1976
1997,4589128191913375,Danuja Wijaya,F,1966
1998,3541839159078527,Panca Sihombing,M,1965


In [71]:
dfPayCards['payCardIDTable'].apply(lambda x: len(str(x))).value_counts()

payCardIDTable
16    1000
15     336
13     178
19     177
12     155
14     154
Name: count, dtype: int64

In [72]:
dfPayCards.insert(1,'payCardBank',np.where(dfPayCards['payCardIDTable'].apply(lambda x: len(str(x))) == 16, 'dki', ''))
dfPayCards['payCardBank'] = np.where(dfPayCards['payCardIDTable'].apply(lambda x: len(str(x))) == 15, 'emoney', dfPayCards['payCardBank'])
dfPayCards['payCardBank'] = np.where(dfPayCards['payCardIDTable'].apply(lambda x: len(str(x))) == 12, 'flazz', dfPayCards['payCardBank'])
dfPayCards['payCardBank'] = np.where(dfPayCards['payCardIDTable'].apply(lambda x: len(str(x))) == 19, 'brizzi', dfPayCards['payCardBank'])
dfPayCards['payCardBank'] = np.where(dfPayCards['payCardIDTable'].apply(lambda x: len(str(x))) == 14, 'bni', dfPayCards['payCardBank'])
dfPayCards['payCardBank'] = np.where(dfPayCards['payCardIDTable'].apply(lambda x: len(str(x))) == 13, 'online', dfPayCards['payCardBank'])
dfPayCards


,payCardIDTable,payCardBank,payCardNameTable,payCardSexTable,payCardBirthDate
0,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011
1,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999
2,213181258600684,emoney,Hj. Tari Narpati,F,1990
3,4616141698869,online,Jaiman Siregar,F,1965
4,501860613964,flazz,Syahrini Nugroho,M,1988
...,...,...,...,...,...
1995,3554847388768390,dki,Pardi Maheswara,F,2010
1996,4303534664855057,dki,Sutan Setya Marpaung,M,1976
1997,4589128191913375,dki,Danuja Wijaya,F,1966
1998,3541839159078527,dki,Panca Sihombing,M,1965


In [73]:
dfPayCards.to_csv('dfPayCards.csv')

# **Create Master for route choice variance**
So we can define usual go and back route for each person

## **Create Master for Go Variance 1**

In [75]:
var1GoCorridorID = []
var1GoDirection = []
var1GoTrip = []
var1GoTapInStops = []
var1GoStopStartSeqList = []
var1GoTapOutStops = []
var1GoStopEndSeqList = []

routeOptions = list(dfRoutes['route_id'].unique())

for _ in range(2000) :
    routeID = rd.choice(routeOptions)
    var1GoCorridorID.append(routeID)
    tripID = rd.choice(dfRoutes[dfRoutes['route_id'] == routeID]['trip_id'].to_list())
    var1GoTrip.append(tripID)
    directionID = dfRoutes[dfRoutes['trip_id'] == tripID]['direction_id'].values[0]
    var1GoDirection.append(directionID)
    stopSeq = dfRoutes[(dfRoutes['trip_id'] == tripID)]['stop_sequence'].sort_values().to_list()
    stopStartSeq = rd.choice(stopSeq[:-1])
    var1GoStopStartSeqList.append(stopStartSeq)
    stopStartID = dfRoutes[(dfRoutes['trip_id'] == tripID) & (dfRoutes['stop_sequence'] == stopStartSeq)]['stop_id'].to_list()[0]
    var1GoTapInStops.append(stopStartID)
    stopEndSeq = rd.choice([seq for seq in stopSeq if seq > stopStartSeq])
    var1GoStopEndSeqList.append(stopEndSeq)
    stopEndID = dfRoutes[(dfRoutes['trip_id'] == tripID) & (dfRoutes['stop_sequence'] == stopEndSeq)]['stop_id'].to_list()[0]
    var1GoTapOutStops.append(stopEndID)

dfPayCards['var1GoCorridorID'] = var1GoCorridorID
dfPayCards['var1GoDirection'] = var1GoDirection
dfPayCards['var1GoTrip'] = var1GoTrip
dfPayCards['var1GoTapInStops'] = var1GoTapInStops
dfPayCards['var1GoStopStartSeqList'] = var1GoStopStartSeqList
dfPayCards['var1GoTapOutStops'] = var1GoTapOutStops
dfPayCards['var1GoStopEndSeqList'] = var1GoStopEndSeqList
dfPayCards

,payCardIDTable,payCardBank,payCardNameTable,payCardSexTable,payCardBirthDate,var1GoCorridorID,var1GoDirection,var1GoTrip,var1GoTapInStops,var1GoStopStartSeqList,var1GoTapOutStops,var1GoStopEndSeqList
0,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,1.0,JAK.84-R04,B01844P,12,B00140P,14
1,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,0.0,6B-L02,B02012P,12,B07459P,27
2,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,1.0,JAK.17-R02,B05725P,0,B04507P,12
3,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,0.0,JAK.25-R04,B01799P,45,B06197P,46
4,501860613964,flazz,Syahrini Nugroho,M,1988,3,1.0,3-R06,G00128,2,G00153,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3554847388768390,dki,Pardi Maheswara,F,2010,JAK.88,0.0,JAK.88-R01,B05986P,34,B05988P,35
1996,4303534664855057,dki,Sutan Setya Marpaung,M,1976,JAK.24,1.0,JAK.24-R02,B00774P,5,B06066P,6
1997,4589128191913375,dki,Danuja Wijaya,F,1966,11B,0.0,11B-R01,B06653P,2,B06652P,3
1998,3541839159078527,dki,Panca Sihombing,M,1965,10H,1.0,10H-R09,G00436,5,G00121,9


## **Create Master for Back Variance 1**

In [76]:
import random

def random_int_exclude(exclude, lower=0, upper=2000):
    while True:
        x = random.randint(lower, upper)
        if x not in exclude:
            return x
emptyTripIdRouteId = []

var1BackCorridorID = []
var1BackDirection = []
var1BackTrip = []
var1BackTapInStops = []
var1BackStopStartSeqList = []
var1BackTapOutStops = []
var1BackStopEndSeqList = []

routeOptions = list(dfRoutes['route_id'].unique())

for i in range(2000) :
    routeID = dfPayCards.iloc[i,5]
    var1BackCorridorID.append(routeID)
    directionID = 0 if dfPayCards.iloc[i,6] == 1 else 1
    var1BackDirection.append(directionID)
    tripID_list = dfRoutes[(dfRoutes['route_id'] == routeID) & (dfRoutes['direction_id'] == directionID)]['trip_id'].to_list()
    while len(tripID_list) == 0 :
        emptyTripIdRouteId.append(i)
        x = random_int_exclude(emptyTripIdRouteId,upper=i)
        routeID = dfPayCards.iloc[x,5]
        directionID = 0 if dfPayCards.iloc[i,6] == 1 else 1
        tripID_list = dfRoutes[(dfRoutes['route_id'] == routeID) & (dfRoutes['direction_id'] == directionID)]['trip_id'].to_list()
    tripID = rd.choice(tripID_list)
    var1BackTrip.append(tripID)
    stopSeq = dfRoutes[(dfRoutes['trip_id'] == tripID)]['stop_sequence'].sort_values().to_list()
    stopStartSeq = rd.choice(stopSeq[:-1])
    var1BackStopStartSeqList.append(stopStartSeq)
    stopStartID = dfRoutes[(dfRoutes['trip_id'] == tripID) & (dfRoutes['stop_sequence'] == stopStartSeq)]['stop_id'].to_list()[0]
    var1BackTapInStops.append(stopStartID)
    stopEndSeq = rd.choice([seq for seq in stopSeq if seq > stopStartSeq])
    var1BackStopEndSeqList.append(stopEndSeq)
    stopEndID = dfRoutes[(dfRoutes['trip_id'] == tripID) & (dfRoutes['stop_sequence'] == stopEndSeq)]['stop_id'].to_list()[0]
    var1BackTapOutStops.append(stopEndID)

dfPayCards['var1BackCorridorID'] = var1BackCorridorID
dfPayCards['var1BackDirection'] = var1BackDirection
dfPayCards['var1BackTrip'] = var1BackTrip
dfPayCards['var1BackTapInStops'] = var1BackTapInStops
dfPayCards['var1BackStopStartSeqList'] = var1BackStopStartSeqList
dfPayCards['var1BackTapOutStops'] = var1BackTapOutStops
dfPayCards['var1BackStopEndSeqList'] = var1BackStopEndSeqList
dfPayCards

,payCardIDTable,payCardBank,payCardNameTable,payCardSexTable,payCardBirthDate,var1GoCorridorID,var1GoDirection,var1GoTrip,var1GoTapInStops,var1GoStopStartSeqList,var1GoTapOutStops,var1GoStopEndSeqList,var1BackCorridorID,var1BackDirection,var1BackTrip,var1BackTapInStops,var1BackStopStartSeqList,var1BackTapOutStops,var1BackStopEndSeqList
0,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,1.0,JAK.84-R04,B01844P,12,B00140P,14,JAK.84,0,JAK.84-R03,B02186P,5,B02189P,15
1,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,0.0,6B-L02,B02012P,12,B07459P,27,6B,1,6B-R07,G00028,4,G00022,5
2,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,1.0,JAK.17-R02,B05725P,0,B04507P,12,JAK.17,0,6B-L02,G00272,9,G00028,22
3,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,0.0,JAK.25-R04,B01799P,45,B06197P,46,JAK.25,1,JAK.25-R06,B01562P,10,B06282P,20
4,501860613964,flazz,Syahrini Nugroho,M,1988,3,1.0,3-R06,G00128,2,G00153,7,3,0,3-L06,G00163,11,G00132,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3554847388768390,dki,Pardi Maheswara,F,2010,JAK.88,0.0,JAK.88-R01,B05986P,34,B05988P,35,JAK.88,1,JAK.88-R02,B06000P,21,B05728P,30
1996,4303534664855057,dki,Sutan Setya Marpaung,M,1976,JAK.24,1.0,JAK.24-R02,B00774P,5,B06066P,6,JAK.24,0,JAK.24-R01,B01897P,17,B02446P,35
1997,4589128191913375,dki,Danuja Wijaya,F,1966,11B,0.0,11B-R01,B06653P,2,B06652P,3,11B,1,11B-R02,B06654P,3,B03291P,5
1998,3541839159078527,dki,Panca Sihombing,M,1965,10H,1.0,10H-R09,G00436,5,G00121,9,10H,0,10H-R07,G00398,9,G00437,10


# **Create a class or function for generating the dummy data transaction**
*in a sequential ways with certain rules*

In [77]:
class TransportProvider(BaseProvider) :
    def __init__(self, generator):
        super().__init__(generator)
        self.dfRoute = self.load_dfRoute()
        self.dfPayCard = self.load_dfPayCard()

    def load_dfRoute(self):
        dfRouter = dfRoutes
        return dfRouter

    def load_dfPayCard(self):
        dfPayCardr = dfPayCards
        return dfPayCardr
    
    def routeDefiner(self,dfcust=None,tanggal=[2023,5,1],waktu=[7,9,2,0],notNA=0,goBack='random') :
        dummy = Faker(locale='id_ID')
        dfRoute = self.dfRoute
        if dfcust == None :
            dfPayCard = self.dfPayCard
        else :
            dfPayCard = self.dfPayCard.iloc[dfcust[0]:dfcust[1]]

        transID = []
        payCardID = []
        payCardBank = []
        payCardName = []
        payCardSex = []
        payCardBirthDate = []
        corridorID = []
        corridorName = []
        direction = []
        tapInStops = []
        tapInStopsName = []
        tapInStopsLat = []
        tapInStopsLon = []
        tapInTime = []
        tapOutStops = []
        tapOutStopsName = []
        tapOutStopsLat = []
        tapOutStopsLon = []
        tapOutTime = []
        payAmount = []
        stopStartSeqList = []
        stopEndSeqList = []
        
        for i in range(len(dfPayCard)) :
            # TRANSid
            transID.append(dummy.pystr_format(string_format='????###?#?##??').upper())

            # PROFILE
            payCardID.append(dfPayCard.iloc[i,0])
            payCardBank.append(dfPayCard.iloc[i,1])
            payCardName.append(dfPayCard.iloc[i,2])
            payCardSex.append(dfPayCard.iloc[i,3])
            payCardBirthDate.append(dfPayCard.iloc[i,4])

            # ROUTE
            if notNA == 0 :
                routeOptions = list(dfRoute['route_id'].unique())
            elif notNA == 1 :
                routeOptions = list(dfRoute[dfRoute['fare_id'].notna()]['route_id'].unique())
            elif notNA == 2 :
                routeOptions = list(dfRoute[dfRoute['fare_id'].isna()]['route_id'].unique())
            elif isinstance(notNA ,str) :
                routeOptions = [notNA]
            if goBack == 'random' :
                routeID = rd.choice(routeOptions)
                tripID = rd.choice(dfRoute[dfRoute['route_id'] == routeID]['trip_id'].to_list())
                directionID = dfRoute[dfRoute['trip_id'] == tripID]['direction_id'].values[0]
                stopSeq = dfRoute[(dfRoute['trip_id'] == tripID)]['stop_sequence'].sort_values().to_list()
                stopStartSeq = rd.choice(stopSeq[:-1])
                stopStartID = dfRoute[(dfRoute['trip_id'] == tripID) & (dfRoute['stop_sequence'] == stopStartSeq)]['stop_id'].to_list()[0]
                stopEndSeq = rd.choice([seq for seq in stopSeq if seq > stopStartSeq])
                stopEndID = dfRoute[(dfRoute['trip_id'] == tripID) & (dfRoute['stop_sequence'] == stopEndSeq)]['stop_id'].to_list()[0]
            elif goBack == 'go' :
                routeID = dfPayCard.iloc[i,5]
                tripID = dfPayCard.iloc[i,7]
                directionID = dfPayCard.iloc[i,6]
                stopStartSeq = dfPayCard.iloc[i,9]
                stopStartID = dfPayCard.iloc[i,8]
                stopEndSeq = dfPayCard.iloc[i,11]
                stopEndID = dfPayCard.iloc[i,10]   
            elif goBack == 'back' :
                routeID = dfPayCard.iloc[i,12]
                tripID = dfPayCard.iloc[i,14]
                directionID = dfPayCard.iloc[i,13]
                stopStartSeq = dfPayCard.iloc[i,-3]
                stopStartID = dfPayCard.iloc[i,-4]
                stopEndSeq = dfPayCard.iloc[i,-1]
                stopEndID = dfPayCard.iloc[i,-2]
            corridorID.append(routeID)
            corridorName.append(dfRoute[dfRoute['route_id'] == routeID]['route_long_name'].values[0])
            direction.append(directionID)
            stopStartSeqList.append(stopStartSeq)
            tapInStops.append(stopStartID)
            tapInStopsName.append(dfRoute[dfRoute['stop_id'] == stopStartID]['stop_name'].values[0])
            tapInStopsLat.append(dfRoute[dfRoute['stop_id'] == stopStartID]['stop_lat'].values[0])
            tapInStopsLon.append(dfRoute[dfRoute['stop_id'] == stopStartID]['stop_lon'].values[0])
            stopEndSeqList.append(stopEndSeq)
            tapOutStops.append(stopEndID)
            tapOutStopsName.append(dfRoute[dfRoute['stop_id'] == stopEndID]['stop_name'].values[0])
            tapOutStopsLat.append(dfRoute[dfRoute['stop_id'] == stopEndID]['stop_lat'].values[0])
            tapOutStopsLon.append(dfRoute[dfRoute['stop_id'] == stopEndID]['stop_lon'].values[0])

            # TIME
            # lastDay = calendar.monthrange(tanggal[0], tanggal[1])[1]
            # day = rd.randint(1, lastDay)
            startDatetime = datetime(tanggal[0], tanggal[1], tanggal[2], waktu[0], 0, 0)
            endDatetime = datetime(tanggal[0], tanggal[1], tanggal[2], waktu[1], 0, 0) - timedelta(seconds=1)
            stopStartTime = dummy.date_time_between_dates(startDatetime, endDatetime)
            tapInTime.append(stopStartTime)
            maxDuration = timedelta(hours=waktu[2])
            minDuration = timedelta(minutes=waktu[3])
            stopEndTime = dummy.date_time_between_dates(stopStartTime + minDuration, stopStartTime + maxDuration)
            tapOutTime.append(stopEndTime)

            # PAYamount
            payAmount.append(dfRoute[dfRoute['route_id'] == routeID]['price'].values[0])

        dfRouteDefiner = pd.DataFrame(zip(transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,
                                          corridorID,corridorName,direction,
                                          tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeqList,tapInTime,
                                          tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeqList,tapOutTime,payAmount),
                                      columns = ['transID','payCardID','payCardBank','payCardName','payCardSex','payCardBirthDate',
                                                 'corridorID','corridorName','direction',
                                                 'tapInStops','tapInStopsName','tapInStopsLat','tapInStopsLon','stopStartSeq','tapInTime',
                                                 'tapOutStops','tapOutStopsName','tapOutStopsLat','tapOutStopsLon','stopEndSeq','tapOutTime','payAmount'])

        return dfRouteDefiner

fake.add_provider(TransportProvider)

In [78]:
# TEST RUN
test = fake.routeDefiner(dfcust=[0,100],tanggal=[2023,4,1],waktu=[5,7,2,60],goBack='back')
display(test,
        test[test['payCardID'] == '6011454241671356'])

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount
0,AONB896H8A19DT,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,0,B02186P,Mall Basura 1,-6.225218,106.878560,5,2023-04-01 05:35:52,B02189P,Mall Cipinang Indah 1,-6.239676,106.894050,15,2023-04-01 06:38:43,0.0
1,UOHN404K0V24WS,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,1,G00028,Tosari,-6.196742,106.823115,4,2023-04-01 06:30:19,G00022,Dukuh Atas 1,-6.205733,106.822284,5,2023-04-01 08:05:06,3500.0
2,SJXH583C6G63IA,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,0,G00272,Mampang Prapatan,-6.242748,106.825685,9,2023-04-01 05:28:50,G00028,Tosari,-6.196742,106.823115,22,2023-04-01 06:47:14,0.0
3,KYPT113X0N66WH,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,1,B01562P,Jln. Saorma Kalisari,-6.338286,106.858750,10,2023-04-01 05:39:13,B06282P,Jln. Kobangdiklat II,-6.326922,106.844680,20,2023-04-01 06:42:56,0.0
4,EBYQ226J5E35HR,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,0,G00163,Petojo,-6.169948,106.816978,11,2023-04-01 05:08:03,G00132,Jelambar,-6.166614,106.786610,19,2023-04-01 07:06:07,3500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,FTQD153Y7Z39WB,4768847421320676029,brizzi,"Zaenab Safitri, S.Pd",M,1971,JAK.93,Jeruk Purut - Kebayoran Lama,0,B07414P,Masjid Jami Darussalam Kebayoran Lama,-6.240536,106.782725,45,2023-04-01 06:54:03,B05455P,St. Kebayoran 2,-6.235723,106.782850,49,2023-04-01 08:42:46,0.0
96,CRTL521C4V89CE,5275691287740990,dki,Juli Kuswoyo,M,2008,JAK.15,Terminal Tanjung Priok - Rusun Marunda,1,B06789P,Jl. Marunda Makmur I,-6.107259,106.959837,48,2023-04-01 06:07:10,B00137P,Bidara 2,-6.106075,106.959420,49,2023-04-01 07:22:43,0.0
97,RJXU460J3Z19EQ,4015324376932,online,Legawa Jailani,M,1983,JAK.77,Tanjung Priok - Jembatan Item,1,B00057P,Apartemen Metro Sunter,-6.133960,106.872080,19,2023-04-01 06:01:34,B05737P,The Mansion Bougenvile,-6.147950,106.855180,34,2023-04-01 07:46:25,0.0
98,SNJZ315D9L03EY,180009892169890,emoney,dr. Ami Riyanti,M,2005,M2,Pulo Gadung - Monas,1,G00042,Monumen Nasional,-6.176252,106.822922,0,2023-04-01 06:03:38,G00698,Pulo Mas,-6.174703,106.892807,13,2023-04-01 07:05:26,3500.0


,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount


# **Creating Transaction Variances**

## 300 People that constantly go and back during weekdays MORNING - EVENING

In [79]:
import random

dfTransaction = pd.DataFrame()
for i in range(3,8) :
    for t,g in zip([[5,7,1,20],[16,18,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        transaction = fake.routeDefiner(dfcust=[0,300],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
for i in range(10,15) :
    for t,g in zip([[5,7,1,20],[16,18,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        transaction = fake.routeDefiner(dfcust=[0,300],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
for i in range(17,22) :
    for t,g in zip([[5,7,1,20],[16,18,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        transaction = fake.routeDefiner(dfcust=[0,300],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
for i in range(24,29) :
    for t,g in zip([[5,7,1,20],[16,18,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        transaction = fake.routeDefiner(dfcust=[0,300],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
        
display(dfTransaction,
        dfTransaction[dfTransaction['payCardID'] == '4996225095064169'])

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount
0,ATBB895U7P81BP,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,1.0,B01844P,Kav Marinir Kalimalang,-6.247975,106.926590,12,2022-06-03 06:11:03,B00140P,Billy Moon,-6.248271,106.930930,14,2022-06-03 06:56:23,0.0
1,NOTZ322M1L09PJ,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,0.0,B02012P,Komdak 1,-6.222333,106.814070,12,2022-06-03 05:50:39,B07459P,Jamsostek 2,-6.233593,106.822976,27,2022-06-03 06:23:31,3500.0
2,HMTO695H4K97TQ,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,1.0,B05725P,Term. Senen,-6.173560,106.842130,0,2022-06-03 06:27:40,B04507P,Sbr. St. Gg. Sentiong,-6.185904,106.850810,12,2022-06-03 07:17:18,0.0
3,PRUZ589F4F05JE,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,0.0,B01799P,Kalisari Lapan Balai Kimia,-6.342833,106.858140,45,2022-06-03 06:47:12,B06197P,Jln. Balai Kimia,-6.343677,106.858020,46,2022-06-03 07:18:45,0.0
4,YYMJ647F6V30WB,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,1.0,G00128,Roxy,-6.166316,106.796808,2,2022-06-03 06:08:59,G00153,Pulo Nangka,-6.154657,106.738095,7,2022-06-03 06:38:50,3500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,ADID255M7N77LG,4034858908265765,dki,"Puti Gawati Santoso, S.H.",M,2012,9C,Pinang Ranti - Bundaran Senayan,1.0,G00418,Pancoran Tugu Arah Timur,-6.243082,106.843920,6,2022-10-28 16:52:19,G00416,Tebet Eco Park Arah Timur,-6.243056,106.852080,7,2022-10-28 18:47:21,3500.0
296,JHVY641I5C64AH,4434860445101023854,brizzi,"Banawa Saefullah, S.H.",M,1982,11P,Rusun Pondok Bambu - Walikota Jakarta Timur,1.0,B03135P,RS Duren Sawit 2,-6.232262,106.910190,22,2022-10-28 17:08:31,B01677P,Jln. Teluk Mandar 2,-6.238944,106.910040,24,2022-10-28 19:01:50,3500.0
297,LCEU241O6V43OO,36948489131522,bni,Citra Uyainah,F,2002,M4,Pulo Gadung - Galunggung,1.0,G00186,Rawamangun,-6.192881,106.880338,9,2022-10-28 17:26:38,G00181,Pemuda Rawamangun,-6.193435,106.891694,12,2022-10-28 18:20:26,3500.0
298,GRDS927T2J31HL,3586081163452283,dki,Putri Pratiwi,F,1989,JAK.71,Kampung Rambutan - Pinang Ranti,0.0,B03051P,Puskesmas Kel. Dukuh,-6.302654,106.880260,6,2022-10-28 17:42:12,B00955P,Jln. Bumi Pratama III,-6.296434,106.878350,12,2022-10-28 18:47:13,0.0


,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount


In [80]:

dfTemp = dfTransaction
# Convert tapInTime to datetime
dfTemp['tapInTime'] = pd.to_datetime(dfTemp['tapInTime'])

# Extract year and month
dfTemp['year'] = dfTemp['tapInTime'].dt.year
dfTemp['month'] = dfTemp['tapInTime'].dt.month

# Group by year and month and count the number of transactions
transaction_counts = dfTemp.groupby(['year', 'month']).size().reset_index(name='count')

print(transaction_counts)

    year  month  count
0   2022      3    300
1   2022      4    600
2   2022      5    600
3   2022      6    900
4   2022      7    600
5   2022      8    600
6   2022      9   1200
7   2022     10    300
8   2022     11    900
9   2022     12    900
10  2023      1    300
11  2023      2    300
12  2023      3   1200
13  2023      4    600
14  2023      5    300
15  2023      6    600
16  2023      7    300
17  2023      9    300
18  2023     10    300
19  2023     11    600
20  2023     12    300


## 500 people that constantly go and back during weekdays LATE MORNING - LATE NIGHT

In [81]:
for i in range(3,8) :
    for t,g in zip([[6,10,2,20],[17,22,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        transaction = fake.routeDefiner(dfcust=[300,800],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
for i in range(10,15) :
    for t,g in zip([[6,10,2,20],[17,22,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        transaction = fake.routeDefiner(dfcust=[300,800],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
for i in range(17,22) :
    for t,g in zip([[6,10,2,20],[17,22,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        transaction = fake.routeDefiner(dfcust=[300,800],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
for i in range(24,29) :
    for t,g in zip([[6,10,2,20],[17,22,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        transaction = fake.routeDefiner(dfcust=[300,800],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
dfTransaction

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,year,month
0,ATBB895U7P81BP,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,1.0,B01844P,Kav Marinir Kalimalang,-6.247975,106.926590,12,2022-06-03 06:11:03,B00140P,Billy Moon,-6.248271,106.930930,14,2022-06-03 06:56:23,0.0,2022.0,6.0
1,NOTZ322M1L09PJ,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,0.0,B02012P,Komdak 1,-6.222333,106.814070,12,2022-06-03 05:50:39,B07459P,Jamsostek 2,-6.233593,106.822976,27,2022-06-03 06:23:31,3500.0,2022.0,6.0
2,HMTO695H4K97TQ,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,1.0,B05725P,Term. Senen,-6.173560,106.842130,0,2022-06-03 06:27:40,B04507P,Sbr. St. Gg. Sentiong,-6.185904,106.850810,12,2022-06-03 07:17:18,0.0,2022.0,6.0
3,PRUZ589F4F05JE,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,0.0,B01799P,Kalisari Lapan Balai Kimia,-6.342833,106.858140,45,2022-06-03 06:47:12,B06197P,Jln. Balai Kimia,-6.343677,106.858020,46,2022-06-03 07:18:45,0.0,2022.0,6.0
4,YYMJ647F6V30WB,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,1.0,G00128,Roxy,-6.166316,106.796808,2,2022-06-03 06:08:59,G00153,Pulo Nangka,-6.154657,106.738095,7,2022-06-03 06:38:50,3500.0,2022.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,SJCU350B1V73XH,30500638743321,bni,Lulut Padmasari,F,1982,JAK.64,Lenteng Agung - Aseli,1.0,B02454P,MIN 8 Srengseng Sawah,-6.335943,106.826280,17,2022-05-28 19:18:51,B01683P,Jln. Timbul IV,-6.350916,106.807710,31,2022-05-28 20:39:10,0.0,NaN,NaN
496,QHNZ645A8F21XR,3548804129929087,dki,Gabriella Firmansyah,F,1988,JAK.99,Term. Pulo Gadung - Lampiri,1.0,B06717P,Jln. Inspeksi Kalimalang Lampiri,-6.249943,106.938978,0,2022-05-28 21:01:14,B06719P,Jln. Rawabadung,-6.208988,106.922724,20,2022-05-28 23:01:14,0.0,NaN,NaN
497,QNCQ110D1R69ME,6569645826174551,dki,Jasmin Santoso,M,2009,JAK.10,Tanah Abang - Kota,0.0,B04649P,Sekolah Pinangsia,-6.138056,106.816070,32,2022-05-28 20:28:00,B04951P,Simpang Kopi Roa Malaka,-6.134185,106.809860,36,2022-05-28 21:56:07,0.0,NaN,NaN
498,HQIW077P9G63TN,345107621908604,emoney,Tira Pudjiastuti,F,2000,2Q,Gondangdia - Balai Kota,1.0,B00304P,Dewan Pers,-6.182996,106.827090,3,2022-05-28 17:11:17,B06053P,Selatan St. Gondangdia,-6.187059,106.832900,8,2022-05-28 18:45:07,3500.0,NaN,NaN


In [82]:

dfTemp = dfTransaction
# Convert tapInTime to datetime
dfTemp['tapInTime'] = pd.to_datetime(dfTemp['tapInTime'])

# Extract year and month
dfTemp['year'] = dfTemp['tapInTime'].dt.year
dfTemp['month'] = dfTemp['tapInTime'].dt.month

# Group by year and month and count the number of transactions
transaction_counts = dfTemp.groupby(['year', 'month']).size().reset_index(name='count')

print(transaction_counts)

    year  month  count
0   2022      1    500
1   2022      2    500
2   2022      3    800
3   2022      4   1100
4   2022      5   1600
5   2022      6   1900
6   2022      7   1100
7   2022      8    600
8   2022      9   2200
9   2022     10   3300
10  2022     11    900
11  2022     12   1400
12  2023      1   1300
13  2023      2    300
14  2023      3   2700
15  2023      4   1100
16  2023      5    800
17  2023      6   1600
18  2023      7   1300
19  2023      8    500
20  2023      9   2800
21  2023     10    800
22  2023     11   1600
23  2023     12   1300


## 700 people once per Weekend

In [84]:
days_in_month = {
    1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
    7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
}

for i,k in zip(range(800,1500),range(801,1501)) :
    dateN = rd.randint(8,9)
    y =  random.choice([2022, 2023])
    m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
    if m == 2 and calendar.isleap(y):
        max_day = 29
    else:
        max_day = days_in_month[m]
    
    dateN = rd.randint(1, max_day)
    transaction = fake.routeDefiner(dfcust=[i,k],tanggal=[y,m,dateN],waktu=[5,22,2,15])
    dfTransaction = pd.concat([dfTransaction,transaction])

for i,k in zip(range(800,1500),range(801,1501)) :
    dateN = rd.randint(15,16)
    y =  random.choice([2022, 2023])
    m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
    if m == 2 and calendar.isleap(y):
        max_day = 29
    else:
        max_day = days_in_month[m]
    
    dateN = rd.randint(1, max_day)
    transaction = fake.routeDefiner(dfcust=[i,k],tanggal=[y,m,dateN],waktu=[5,22,2,15])
    dfTransaction = pd.concat([dfTransaction,transaction])

for i,k in zip(range(800,1500),range(801,1501)) :
    dateN = rd.randint(22,23)
    y =  random.choice([2022, 2023])
    m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
    if m == 2 and calendar.isleap(y):
        max_day = 29
    else:
        max_day = days_in_month[m]
    
    dateN = rd.randint(1, max_day)
    transaction = fake.routeDefiner(dfcust=[i,k],tanggal=[y,m,dateN],waktu=[5,22,2,15])
    dfTransaction = pd.concat([dfTransaction,transaction])

for i,k in zip(range(800,1500),range(801,1501)) :
    dateN = rd.randint(29,30)
    y =  random.choice([2022, 2023])
    m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
    if m == 2 and calendar.isleap(y):
        max_day = 29
    else:
        max_day = days_in_month[m]
    
    dateN = rd.randint(1, max_day)

    transaction = fake.routeDefiner(dfcust=[i,k],tanggal=[y,m,dateN],waktu=[5,22,2,15])
    dfTransaction = pd.concat([dfTransaction,transaction])

dfTransaction

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,year,month
0,ATBB895U7P81BP,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,1.0,B01844P,Kav Marinir Kalimalang,-6.247975,106.926590,12,2022-06-03 06:11:03,B00140P,Billy Moon,-6.248271,106.930930,14,2022-06-03 06:56:23,0.0,2022.0,6.0
1,NOTZ322M1L09PJ,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,0.0,B02012P,Komdak 1,-6.222333,106.814070,12,2022-06-03 05:50:39,B07459P,Jamsostek 2,-6.233593,106.822976,27,2022-06-03 06:23:31,3500.0,2022.0,6.0
2,HMTO695H4K97TQ,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,1.0,B05725P,Term. Senen,-6.173560,106.842130,0,2022-06-03 06:27:40,B04507P,Sbr. St. Gg. Sentiong,-6.185904,106.850810,12,2022-06-03 07:17:18,0.0,2022.0,6.0
3,PRUZ589F4F05JE,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,0.0,B01799P,Kalisari Lapan Balai Kimia,-6.342833,106.858140,45,2022-06-03 06:47:12,B06197P,Jln. Balai Kimia,-6.343677,106.858020,46,2022-06-03 07:18:45,0.0,2022.0,6.0
4,YYMJ647F6V30WB,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,1.0,G00128,Roxy,-6.166316,106.796808,2,2022-06-03 06:08:59,G00153,Pulo Nangka,-6.154657,106.738095,7,2022-06-03 06:38:50,3500.0,2022.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,PYGA736X9N99BO,6579862505615527,dki,"Kani Budiyanto, S.Pd",M,1971,JAK.100,Term. Pulo Gebang - Rusun Pinus Elok,1.0,B07455P,Jln. Dr. Sumarno,-6.212320,106.943325,32,2023-03-29 16:17:45,G00684,Term. Pulo Gebang,-6.212669,106.953183,35,2023-03-29 17:04:04,0.0,NaN,NaN
0,CAMN933D3S09UL,4446152896480897,dki,Ir. Tari Yuniar,F,1996,JAK.46,Pasar Minggu - Jagakarsa,0.0,B04341P,Sbr. SDN Jagakarsa 01,-6.328227,106.820210,9,2022-03-06 16:00:39,B03420P,SDN Ragunan 14,-6.298262,106.826340,22,2022-03-06 17:27:24,0.0,NaN,NaN
0,TBZT778K8H37WK,213194375425547,emoney,"Dina Hidayanto, S.I.Kom",F,1985,JAK.80,Rawa Buaya - Rawa Kompeni,0.0,B04127P,Sbr. Panti Sosial Bina Laras 1,-6.146651,106.718040,8,2022-09-27 10:25:54,B02370P,Masjid Nurul Istiqomah Pegadungan,-6.129558,106.704370,15,2022-09-27 10:49:19,0.0,NaN,NaN
0,GSJK588U2D27OO,676158027174,flazz,Puji Nugroho,F,1994,JAK.26,Rawamangun - Duren Sawit,1.0,B00527P,Gg. Padi Pelita Cipinang,-6.211832,106.887700,19,2022-05-15 09:43:28,B03135P,RS Duren Sawit 2,-6.232262,106.910190,49,2022-05-15 09:59:55,0.0,NaN,NaN


In [85]:

dfTemp = dfTransaction
# Convert tapInTime to datetime
dfTemp['tapInTime'] = pd.to_datetime(dfTemp['tapInTime'])

# Extract year and month
dfTemp['year'] = dfTemp['tapInTime'].dt.year
dfTemp['month'] = dfTemp['tapInTime'].dt.month

# Group by year and month and count the number of transactions
transaction_counts = dfTemp.groupby(['year', 'month']).size().reset_index(name='count')

print(transaction_counts)

    year  month  count
0   2022      1    698
1   2022      2    712
2   2022      3   1000
3   2022      4   1305
4   2022      5   1810
5   2022      6   2097
6   2022      7   1285
7   2022      8    820
8   2022      9   2416
9   2022     10   3510
10  2022     11   1120
11  2022     12   1601
12  2023      1   1477
13  2023      2    504
14  2023      3   2919
15  2023      4   1305
16  2023      5    992
17  2023      6   1796
18  2023      7   1508
19  2023      8    698
20  2023      9   3004
21  2023     10   1016
22  2023     11   1808
23  2023     12   1521


## 200 random people anytime 2x go and back everyday at mid of the month

In [86]:
for i in range(13,20) :
    for t,g in zip([[5,14,2,20],[15,21,2,50]],['go','back']) :
        y =  random.choice([2022, 2023])
        m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
        if m == 2 and calendar.isleap(y):
            max_day = 29
        else:
            max_day = days_in_month[m]
        transaction = fake.routeDefiner(dfcust=[1500,1700],tanggal=[y,m,i],waktu=t,goBack=g)
        dfTransaction = pd.concat([dfTransaction,transaction])
dfTransaction

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,year,month
0,ATBB895U7P81BP,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,1.0,B01844P,Kav Marinir Kalimalang,-6.247975,106.926590,12,2022-06-03 06:11:03,B00140P,Billy Moon,-6.248271,106.930930,14,2022-06-03 06:56:23,0.0,2022.0,6.0
1,NOTZ322M1L09PJ,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,0.0,B02012P,Komdak 1,-6.222333,106.814070,12,2022-06-03 05:50:39,B07459P,Jamsostek 2,-6.233593,106.822976,27,2022-06-03 06:23:31,3500.0,2022.0,6.0
2,HMTO695H4K97TQ,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,1.0,B05725P,Term. Senen,-6.173560,106.842130,0,2022-06-03 06:27:40,B04507P,Sbr. St. Gg. Sentiong,-6.185904,106.850810,12,2022-06-03 07:17:18,0.0,2022.0,6.0
3,PRUZ589F4F05JE,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,0.0,B01799P,Kalisari Lapan Balai Kimia,-6.342833,106.858140,45,2022-06-03 06:47:12,B06197P,Jln. Balai Kimia,-6.343677,106.858020,46,2022-06-03 07:18:45,0.0,2022.0,6.0
4,YYMJ647F6V30WB,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,1.0,G00128,Roxy,-6.166316,106.796808,2,2022-06-03 06:08:59,G00153,Pulo Nangka,-6.154657,106.738095,7,2022-06-03 06:38:50,3500.0,2022.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,GYVC350B2Y97NN,6011656242555376,dki,Endra Hastuti,F,1989,JAK.90,Terminal Tanjung Priok - Rusun Kemayoran,1.0,B05960P,Sbr. Jln. Samudera 1,-6.116370,106.870550,37,2022-06-19 15:38:56,B05956P,Depo Kontainer Tj. Priok 2,-6.111530,106.876940,40,2022-06-19 17:10:52,0.0,NaN,NaN
196,JDEO109R0H36KQ,379585112126738,emoney,"Darmana Sihombing, S.Ked",F,2012,S11,BSD - Jelambar,1.0,B01736P,Jelambar 2,-6.166706,106.786430,0,2022-06-19 16:45:21,B03123P,RS Asshobirin 2,-6.258481,106.651990,16,2022-06-19 18:14:30,3500.0,NaN,NaN
197,WJQM762B8W28JB,3518520206947380,dki,Irsad Suryatmi,F,1984,JAK.16,Cililitan - Condet,0.0,B01038P,Jln. Deri Ayama 1,-6.272384,106.866850,6,2022-06-19 15:36:51,B04999P,Simpang Masjid Condet Batu Ampar,-6.280831,106.860240,14,2022-06-19 17:32:29,0.0,NaN,NaN
198,QDRA264L4B73AK,630440423535,flazz,Nadine Mansur,F,1986,1P,Senen - Blok M,0.0,B02680P,Ps. Senen Jaya,-6.177735,106.842160,5,2022-06-19 19:09:52,B00083P,Atrium 1,-6.177759,106.841200,6,2022-06-19 20:59:40,3500.0,NaN,NaN


## 300 random people anytime once

In [87]:
for i,k in zip(range(1700,2000),range(1701,2001)) :
    dateN = rd.randint(1,30)
    y =  random.choice([2022, 2023])
    m = random.choice([1,2,3,4,5,6,7,8,9,10,11,12])
    if m == 2 and calendar.isleap(y):
        max_day = 29
    else:
        max_day = days_in_month[m]
    transaction = fake.routeDefiner(dfcust=[i,k],tanggal=[2023,4,dateN],waktu=[5,22,3,20])
    dfTransaction = pd.concat([dfTransaction,transaction])
dfTransaction

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,year,month
0,ATBB895U7P81BP,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,1.0,B01844P,Kav Marinir Kalimalang,-6.247975,106.926590,12,2022-06-03 06:11:03,B00140P,Billy Moon,-6.248271,106.930930,14,2022-06-03 06:56:23,0.0,2022.0,6.0
1,NOTZ322M1L09PJ,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,0.0,B02012P,Komdak 1,-6.222333,106.814070,12,2022-06-03 05:50:39,B07459P,Jamsostek 2,-6.233593,106.822976,27,2022-06-03 06:23:31,3500.0,2022.0,6.0
2,HMTO695H4K97TQ,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,1.0,B05725P,Term. Senen,-6.173560,106.842130,0,2022-06-03 06:27:40,B04507P,Sbr. St. Gg. Sentiong,-6.185904,106.850810,12,2022-06-03 07:17:18,0.0,2022.0,6.0
3,PRUZ589F4F05JE,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,0.0,B01799P,Kalisari Lapan Balai Kimia,-6.342833,106.858140,45,2022-06-03 06:47:12,B06197P,Jln. Balai Kimia,-6.343677,106.858020,46,2022-06-03 07:18:45,0.0,2022.0,6.0
4,YYMJ647F6V30WB,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,1.0,G00128,Roxy,-6.166316,106.796808,2,2022-06-03 06:08:59,G00153,Pulo Nangka,-6.154657,106.738095,7,2022-06-03 06:38:50,3500.0,2022.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,URZA722W6G73UX,3554847388768390,dki,Pardi Maheswara,F,2010,M7,Kampung Rambutan - Kampung Melayu,0.0,G00321,Kramat Jati,-6.268810,106.866589,5,2023-04-03 21:02:08,G00337,BNN Arah Barat,-6.246687,106.873246,9,2023-04-03 22:27:28,3500.0,NaN,NaN
0,QCQM167H9N15GB,4303534664855057,dki,Sutan Setya Marpaung,M,1976,3B,Rusun Flamboyan - Cengkareng,0.0,B04902P,Simpang Kapuk Cengkareng,-6.134300,106.730510,3,2023-04-07 20:45:13,B02900P,Polsek Cengkareng 1,-6.146466,106.728810,9,2023-04-07 21:12:57,3500.0,NaN,NaN
0,VPWK735Y3S03PE,4589128191913375,dki,Danuja Wijaya,F,1966,JAK.10,Tanah Abang - Kota,1.0,B04949P,Simpang Kopi Bandengan Selatan,-6.135839,106.806890,4,2023-04-15 08:22:31,B04799P,Simpang Dinkes,-6.170376,106.815430,33,2023-04-15 09:41:10,0.0,NaN,NaN
0,ORHI960S0X61UU,3541839159078527,dki,Panca Sihombing,M,1965,7B,Kampung Rambutan - Blok M,0.0,B00201P,Budiwarman 1,-6.279071,106.870190,13,2023-04-20 21:35:39,B03173P,RS Restu Kasih,-6.266060,106.865980,58,2023-04-20 21:55:58,3500.0,NaN,NaN


In [88]:
dfTransaction.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40022 entries, 0 to 0
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transID           40022 non-null  object        
 1   payCardID         40022 non-null  object        
 2   payCardBank       40022 non-null  object        
 3   payCardName       40022 non-null  object        
 4   payCardSex        40022 non-null  object        
 5   payCardBirthDate  40022 non-null  int64         
 6   corridorID        40022 non-null  object        
 7   corridorName      40022 non-null  object        
 8   direction         40022 non-null  float64       
 9   tapInStops        40022 non-null  object        
 10  tapInStopsName    40022 non-null  object        
 11  tapInStopsLat     40022 non-null  float64       
 12  tapInStopsLon     40022 non-null  float64       
 13  stopStartSeq      40022 non-null  int64         
 14  tapInTime         40022 non-nul

# **Creating randomize NaN Object**

In [89]:
dfTransNA = dfTransaction.copy()
dfTransNA.reset_index(drop=True,inplace=True)
dfTransNA

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,year,month
0,ATBB895U7P81BP,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,1.0,B01844P,Kav Marinir Kalimalang,-6.247975,106.926590,12,2022-06-03 06:11:03,B00140P,Billy Moon,-6.248271,106.930930,14,2022-06-03 06:56:23,0.0,2022.0,6.0
1,NOTZ322M1L09PJ,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,0.0,B02012P,Komdak 1,-6.222333,106.814070,12,2022-06-03 05:50:39,B07459P,Jamsostek 2,-6.233593,106.822976,27,2022-06-03 06:23:31,3500.0,2022.0,6.0
2,HMTO695H4K97TQ,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,1.0,B05725P,Term. Senen,-6.173560,106.842130,0,2022-06-03 06:27:40,B04507P,Sbr. St. Gg. Sentiong,-6.185904,106.850810,12,2022-06-03 07:17:18,0.0,2022.0,6.0
3,PRUZ589F4F05JE,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,0.0,B01799P,Kalisari Lapan Balai Kimia,-6.342833,106.858140,45,2022-06-03 06:47:12,B06197P,Jln. Balai Kimia,-6.343677,106.858020,46,2022-06-03 07:18:45,0.0,2022.0,6.0
4,YYMJ647F6V30WB,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,1.0,G00128,Roxy,-6.166316,106.796808,2,2022-06-03 06:08:59,G00153,Pulo Nangka,-6.154657,106.738095,7,2022-06-03 06:38:50,3500.0,2022.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40017,URZA722W6G73UX,3554847388768390,dki,Pardi Maheswara,F,2010,M7,Kampung Rambutan - Kampung Melayu,0.0,G00321,Kramat Jati,-6.268810,106.866589,5,2023-04-03 21:02:08,G00337,BNN Arah Barat,-6.246687,106.873246,9,2023-04-03 22:27:28,3500.0,NaN,NaN
40018,QCQM167H9N15GB,4303534664855057,dki,Sutan Setya Marpaung,M,1976,3B,Rusun Flamboyan - Cengkareng,0.0,B04902P,Simpang Kapuk Cengkareng,-6.134300,106.730510,3,2023-04-07 20:45:13,B02900P,Polsek Cengkareng 1,-6.146466,106.728810,9,2023-04-07 21:12:57,3500.0,NaN,NaN
40019,VPWK735Y3S03PE,4589128191913375,dki,Danuja Wijaya,F,1966,JAK.10,Tanah Abang - Kota,1.0,B04949P,Simpang Kopi Bandengan Selatan,-6.135839,106.806890,4,2023-04-15 08:22:31,B04799P,Simpang Dinkes,-6.170376,106.815430,33,2023-04-15 09:41:10,0.0,NaN,NaN
40020,ORHI960S0X61UU,3541839159078527,dki,Panca Sihombing,M,1965,7B,Kampung Rambutan - Blok M,0.0,B00201P,Budiwarman 1,-6.279071,106.870190,13,2023-04-20 21:35:39,B03173P,RS Restu Kasih,-6.266060,106.865980,58,2023-04-20 21:55:58,3500.0,NaN,NaN


In [90]:
dfTransNA['tapInTime'] = dfTransNA['tapInTime'].astype('object')
dfTransNA['tapOutTime'] = dfTransNA['tapOutTime'].astype('object')
display(dfTransNA.info(),
        dfTransNA)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40022 entries, 0 to 40021
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transID           40022 non-null  object 
 1   payCardID         40022 non-null  object 
 2   payCardBank       40022 non-null  object 
 3   payCardName       40022 non-null  object 
 4   payCardSex        40022 non-null  object 
 5   payCardBirthDate  40022 non-null  int64  
 6   corridorID        40022 non-null  object 
 7   corridorName      40022 non-null  object 
 8   direction         40022 non-null  float64
 9   tapInStops        40022 non-null  object 
 10  tapInStopsName    40022 non-null  object 
 11  tapInStopsLat     40022 non-null  float64
 12  tapInStopsLon     40022 non-null  float64
 13  stopStartSeq      40022 non-null  int64  
 14  tapInTime         40022 non-null  object 
 15  tapOutStops       40022 non-null  object 
 16  tapOutStopsName   40022 non-null  object

None

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,year,month
0,ATBB895U7P81BP,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,1.0,B01844P,Kav Marinir Kalimalang,-6.247975,106.926590,12,2022-06-03 06:11:03,B00140P,Billy Moon,-6.248271,106.930930,14,2022-06-03 06:56:23,0.0,2022.0,6.0
1,NOTZ322M1L09PJ,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,0.0,B02012P,Komdak 1,-6.222333,106.814070,12,2022-06-03 05:50:39,B07459P,Jamsostek 2,-6.233593,106.822976,27,2022-06-03 06:23:31,3500.0,2022.0,6.0
2,HMTO695H4K97TQ,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,1.0,B05725P,Term. Senen,-6.173560,106.842130,0,2022-06-03 06:27:40,B04507P,Sbr. St. Gg. Sentiong,-6.185904,106.850810,12,2022-06-03 07:17:18,0.0,2022.0,6.0
3,PRUZ589F4F05JE,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,0.0,B01799P,Kalisari Lapan Balai Kimia,-6.342833,106.858140,45,2022-06-03 06:47:12,B06197P,Jln. Balai Kimia,-6.343677,106.858020,46,2022-06-03 07:18:45,0.0,2022.0,6.0
4,YYMJ647F6V30WB,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,1.0,G00128,Roxy,-6.166316,106.796808,2,2022-06-03 06:08:59,G00153,Pulo Nangka,-6.154657,106.738095,7,2022-06-03 06:38:50,3500.0,2022.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40017,URZA722W6G73UX,3554847388768390,dki,Pardi Maheswara,F,2010,M7,Kampung Rambutan - Kampung Melayu,0.0,G00321,Kramat Jati,-6.268810,106.866589,5,2023-04-03 21:02:08,G00337,BNN Arah Barat,-6.246687,106.873246,9,2023-04-03 22:27:28,3500.0,NaN,NaN
40018,QCQM167H9N15GB,4303534664855057,dki,Sutan Setya Marpaung,M,1976,3B,Rusun Flamboyan - Cengkareng,0.0,B04902P,Simpang Kapuk Cengkareng,-6.134300,106.730510,3,2023-04-07 20:45:13,B02900P,Polsek Cengkareng 1,-6.146466,106.728810,9,2023-04-07 21:12:57,3500.0,NaN,NaN
40019,VPWK735Y3S03PE,4589128191913375,dki,Danuja Wijaya,F,1966,JAK.10,Tanah Abang - Kota,1.0,B04949P,Simpang Kopi Bandengan Selatan,-6.135839,106.806890,4,2023-04-15 08:22:31,B04799P,Simpang Dinkes,-6.170376,106.815430,33,2023-04-15 09:41:10,0.0,NaN,NaN
40020,ORHI960S0X61UU,3541839159078527,dki,Panca Sihombing,M,1965,7B,Kampung Rambutan - Blok M,0.0,B00201P,Budiwarman 1,-6.279071,106.870190,13,2023-04-20 21:35:39,B03173P,RS Restu Kasih,-6.266060,106.865980,58,2023-04-20 21:55:58,3500.0,NaN,NaN


In [91]:
for i in range(1143) :
    a = np.random.randint(0,len(dfTransNA))
    dfTransNA.loc[a,'corridorName'] = np.nan
    dfTransNA.loc[a,'corridorID'] = np.nan
dfTransNA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40022 entries, 0 to 40021
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transID           40022 non-null  object 
 1   payCardID         40022 non-null  object 
 2   payCardBank       40022 non-null  object 
 3   payCardName       40022 non-null  object 
 4   payCardSex        40022 non-null  object 
 5   payCardBirthDate  40022 non-null  int64  
 6   corridorID        38892 non-null  object 
 7   corridorName      38892 non-null  object 
 8   direction         40022 non-null  float64
 9   tapInStops        40022 non-null  object 
 10  tapInStopsName    40022 non-null  object 
 11  tapInStopsLat     40022 non-null  float64
 12  tapInStopsLon     40022 non-null  float64
 13  stopStartSeq      40022 non-null  int64  
 14  tapInTime         40022 non-null  object 
 15  tapOutStops       40022 non-null  object 
 16  tapOutStopsName   40022 non-null  object

In [92]:
for i in range(138) :
    a = np.random.randint(0,len(dfTransNA))
    dfTransNA.loc[a,'corridorID'] = np.nan
for i in range(831) :
    a = np.random.randint(0,len(dfTransNA))
    dfTransNA.loc[a,'corridorName'] = np.nan
dfTransNA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40022 entries, 0 to 40021
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transID           40022 non-null  object 
 1   payCardID         40022 non-null  object 
 2   payCardBank       40022 non-null  object 
 3   payCardName       40022 non-null  object 
 4   payCardSex        40022 non-null  object 
 5   payCardBirthDate  40022 non-null  int64  
 6   corridorID        38758 non-null  object 
 7   corridorName      38091 non-null  object 
 8   direction         40022 non-null  float64
 9   tapInStops        40022 non-null  object 
 10  tapInStopsName    40022 non-null  object 
 11  tapInStopsLat     40022 non-null  float64
 12  tapInStopsLon     40022 non-null  float64
 13  stopStartSeq      40022 non-null  int64  
 14  tapInTime         40022 non-null  object 
 15  tapOutStops       40022 non-null  object 
 16  tapOutStopsName   40022 non-null  object

In [93]:
for i in range(1234) :
    a = np.random.randint(0,len(dfTransNA))
    dfTransNA.loc[a,'tapInStops'] = np.nan
for i in range(991) :
    a = np.random.randint(0,len(dfTransNA))
    dfTransNA.loc[a,'tapOutStops'] = np.nan
dfTransNA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40022 entries, 0 to 40021
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transID           40022 non-null  object 
 1   payCardID         40022 non-null  object 
 2   payCardBank       40022 non-null  object 
 3   payCardName       40022 non-null  object 
 4   payCardSex        40022 non-null  object 
 5   payCardBirthDate  40022 non-null  int64  
 6   corridorID        38758 non-null  object 
 7   corridorName      38091 non-null  object 
 8   direction         40022 non-null  float64
 9   tapInStops        38806 non-null  object 
 10  tapInStopsName    40022 non-null  object 
 11  tapInStopsLat     40022 non-null  float64
 12  tapInStopsLon     40022 non-null  float64
 13  stopStartSeq      40022 non-null  int64  
 14  tapInTime         40022 non-null  object 
 15  tapOutStops       39039 non-null  object 
 16  tapOutStopsName   40022 non-null  object

In [94]:
# for i in range(1234) :
#     a = np.random.randint(0,len(dfTransNA))
#     dfTransNA.loc[a,'tapInStops'] = np.nan
#     dfTransNA.loc[a,'tapInStopsName'] = np.nan
#     dfTransNA.loc[a,'tapInStopsLat'] = np.nan
#     dfTransNA.loc[a,'tapInStopsLon'] = np.nan
#     dfTransNA.loc[a,'stopStartSeq'] = np.nan
for i in range(1362) :
    a = np.random.randint(0,len(dfTransNA))
    dfTransNA.loc[a,'tapOutStops'] = np.nan
    dfTransNA.loc[a,'tapOutStopsName'] = np.nan
    dfTransNA.loc[a,'tapOutStopsLat'] = np.nan
    dfTransNA.loc[a,'tapOutStopsLon'] = np.nan
    dfTransNA.loc[a,'stopEndSeq'] = np.nan
    dfTransNA.loc[a,'tapOutTime'] = np.nan
dfTransNA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40022 entries, 0 to 40021
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transID           40022 non-null  object 
 1   payCardID         40022 non-null  object 
 2   payCardBank       40022 non-null  object 
 3   payCardName       40022 non-null  object 
 4   payCardSex        40022 non-null  object 
 5   payCardBirthDate  40022 non-null  int64  
 6   corridorID        38758 non-null  object 
 7   corridorName      38091 non-null  object 
 8   direction         40022 non-null  float64
 9   tapInStops        38806 non-null  object 
 10  tapInStopsName    40022 non-null  object 
 11  tapInStopsLat     40022 non-null  float64
 12  tapInStopsLon     40022 non-null  float64
 13  stopStartSeq      40022 non-null  int64  
 14  tapInTime         40022 non-null  object 
 15  tapOutStops       37738 non-null  object 
 16  tapOutStopsName   38682 non-null  object

In [95]:
# Check total row that can be used if user choose to dropna all
dfTransNA.notna().all(axis=1).sum()

31992

In [96]:
dfTransNA['transID'].nunique()

40022

In [98]:
# dfTransaction.to_csv('dfTransjakartaV1Full.csv',index=False)
dfTransNA.to_csv('dfTransjakarta2.csv',index=False)


In [99]:
dfTemp = dfTransNA
# Convert tapInTime to datetime
dfTemp['tapInTime'] = pd.to_datetime(dfTemp['tapInTime'])

# Extract year and month
dfTemp['year'] = dfTemp['tapInTime'].dt.year
dfTemp['month'] = dfTemp['tapInTime'].dt.month

# Group by year and month and count the number of transactions
transaction_counts = dfTemp.groupby(['year', 'month']).size().reset_index(name='count')

print(transaction_counts)

    year  month  count
0   2022      1    698
1   2022      2    712
2   2022      3   1000
3   2022      4   1305
4   2022      5   2210
5   2022      6   2897
6   2022      7   1285
7   2022      8   1220
8   2022      9   2616
9   2022     10   3710
10  2022     11   1120
11  2022     12   1601
12  2023      1   1477
13  2023      2    704
14  2023      3   2919
15  2023      4   1605
16  2023      5    992
17  2023      6   1796
18  2023      7   1508
19  2023      8    898
20  2023      9   3404
21  2023     10   1016
22  2023     11   1808
23  2023     12   1521


In [100]:
dfTransNA

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,tapInStopsName,tapInStopsLat,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,year,month
0,ATBB895U7P81BP,4368520028973555,dki,"H. Asman Prakasa, M.Farm",M,2011,JAK.84,Terminal Kampung Melayu - Kapin Raya,1.0,B01844P,Kav Marinir Kalimalang,-6.247975,106.926590,12,2022-06-03 06:11:03,B00140P,Billy Moon,-6.248271,106.930930,14.0,2022-06-03 06:56:23,0.0,2022,6
1,NOTZ322M1L09PJ,5343809282239143,dki,"Mahesa Pratiwi, S.Sos",F,1999,6B,Ragunan - Balai Kota via Semanggi,0.0,B02012P,Komdak 1,-6.222333,106.814070,12,2022-06-03 05:50:39,NaN,NaN,NaN,NaN,NaN,NaN,3500.0,2022,6
2,HMTO695H4K97TQ,213181258600684,emoney,Hj. Tari Narpati,F,1990,JAK.17,Senen - Pulo Gadung,1.0,B05725P,Term. Senen,-6.173560,106.842130,0,2022-06-03 06:27:40,B04507P,Sbr. St. Gg. Sentiong,-6.185904,106.850810,12.0,2022-06-03 07:17:18,0.0,2022,6
3,PRUZ589F4F05JE,4616141698869,online,Jaiman Siregar,F,1965,JAK.25,Pasar Rebo - Kalisari,0.0,B01799P,Kalisari Lapan Balai Kimia,-6.342833,106.858140,45,2022-06-03 06:47:12,B06197P,Jln. Balai Kimia,-6.343677,106.858020,46.0,2022-06-03 07:18:45,0.0,2022,6
4,YYMJ647F6V30WB,501860613964,flazz,Syahrini Nugroho,M,1988,3,Kalideres - Monas via Veteran,1.0,G00128,Roxy,-6.166316,106.796808,2,2022-06-03 06:08:59,G00153,Pulo Nangka,-6.154657,106.738095,7.0,2022-06-03 06:38:50,3500.0,2022,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40017,URZA722W6G73UX,3554847388768390,dki,Pardi Maheswara,F,2010,M7,Kampung Rambutan - Kampung Melayu,0.0,G00321,Kramat Jati,-6.268810,106.866589,5,2023-04-03 21:02:08,G00337,BNN Arah Barat,-6.246687,106.873246,9.0,2023-04-03 22:27:28,3500.0,2023,4
40018,QCQM167H9N15GB,4303534664855057,dki,Sutan Setya Marpaung,M,1976,3B,Rusun Flamboyan - Cengkareng,0.0,B04902P,Simpang Kapuk Cengkareng,-6.134300,106.730510,3,2023-04-07 20:45:13,B02900P,Polsek Cengkareng 1,-6.146466,106.728810,9.0,2023-04-07 21:12:57,3500.0,2023,4
40019,VPWK735Y3S03PE,4589128191913375,dki,Danuja Wijaya,F,1966,JAK.10,Tanah Abang - Kota,1.0,NaN,Simpang Kopi Bandengan Selatan,-6.135839,106.806890,4,2023-04-15 08:22:31,B04799P,Simpang Dinkes,-6.170376,106.815430,33.0,2023-04-15 09:41:10,0.0,2023,4
40020,ORHI960S0X61UU,3541839159078527,dki,Panca Sihombing,M,1965,7B,Kampung Rambutan - Blok M,0.0,B00201P,Budiwarman 1,-6.279071,106.870190,13,2023-04-20 21:35:39,B03173P,RS Restu Kasih,-6.266060,106.865980,58.0,2023-04-20 21:55:58,3500.0,2023,4
